In [1]:
import numpy as np
import pandas as pd

In [69]:
pip install pydotplus

     ---------------------------------------- 0.0/278.7 kB ? eta -:--:--
     -------------------------------------- 278.7/278.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24578 sha256=736348cf4a6d97c2ca8be6d56b1173b77710c4fa623ae2edd53f4d665e66f6ec
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\bd\ce\e8\ff9d9c699514922f57caa22fbd55b0a32761114b4c4acc9e03
Successfully built pydotplus
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pwd

'C:\\Users\\Admin\\capstone projects'

In [5]:
%cd C:\Users\Admin\capstone projects\Network instrusion detection\NSL_Dataset

C:\Users\Admin\capstone projects\Network instrusion detection\NSL_Dataset


In [6]:
train_test = pd.read_csv('train_test.csv')
attack_master = pd.read_csv('attack_master.csv')

In [11]:
train_test_dict = dict(train_test.dtypes)
train_test_dict

{'duration': dtype('int64'),
 'protocol_type': dtype('O'),
 'service': dtype('O'),
 'flag': dtype('O'),
 'src_bytes': dtype('int64'),
 'dst_bytes': dtype('int64'),
 'land': dtype('int64'),
 ' wrong_fragment': dtype('int64'),
 'urgent': dtype('int64'),
 'hot': dtype('int64'),
 'num_failed_logins': dtype('int64'),
 'logged_in': dtype('int64'),
 ' num_compromised': dtype('int64'),
 'root_shell': dtype('int64'),
 'su_attempted': dtype('int64'),
 'num_root': dtype('int64'),
 'num_file_creations': dtype('int64'),
 ' num_shells': dtype('int64'),
 'num_access_files': dtype('int64'),
 'num_outbound_cmds': dtype('int64'),
 'is_host_login': dtype('int64'),
 ' is_guest_login': dtype('int64'),
 'count': dtype('int64'),
 'srv_count': dtype('int64'),
 'serror_rate': dtype('float64'),
 ' srv_serror_rate': dtype('float64'),
 ' rerror_rate': dtype('float64'),
 'srv_rerror_rate': dtype('float64'),
 'same_srv_rate': dtype('float64'),
 'diff_srv_rate': dtype('float64'),
 'srv_diff_host_rate': dtype('float6

In [12]:
train_test.shape

(148517, 44)

In [13]:
train_test1 = train_test[train_test.duplicated() == False]

In [14]:
# to get the dataframe with numeric data
def df_split_num(a,b):
    cat_num = []
    for val in b:
        if b[val] in ['float64','float32','int64','int32']:
            cat_num.append(val)
    train_test_num_df = a[cat_num]
    return train_test_num_df

In [16]:
train_test_num=df_split_num(train_test1,train_test_dict)

In [17]:
# to get the dataframe with categorical data
def df_split_var(a,b):
    cat_var = []
    for val in b:
        if b[val] in ['O']:
            cat_var.append(val)
    train_test_var_df = a[cat_var]
        
    for a in train_test_var_df.columns:
        train_test_var_df[a] = train_test_var_df[a].fillna('missing')
    return train_test_var_df

In [18]:
train_test_var=df_split_var(train_test1,train_test_dict)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17312\1626308036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_var_df[a] = train_test_var_df[a].fillna('missing')


In [19]:
# to update missing value
def missings_value(y):
    y=y.fillna(y.mean())
    return y

In [20]:
train_test_num=train_test_num.apply(lambda y : missings_value(y))

In [21]:
# to identify outliers and filling with 0.01 and 0.99 percentiles
def handling_outliers(y):
    y=y.clip(upper = y.quantile(0.99))
    y=y.clip(lower = y.quantile(0.01))
    return y

In [22]:
#to create dummies of categorical variables and then remove the original variable and dummy count -1
def handling_dummies(df,col_name):
    col_dummies = pd.get_dummies(df[col_name],prefix=col_name)
    col_dummies.drop(col_dummies.columns[0],axis=1,inplace=True)
    df=pd.concat([df,col_dummies],axis=1)
    df.drop(col_name,axis=1,inplace=True)
    return df

In [23]:
train_test_cat = train_test1[train_test_var.columns.difference(['attack','train_test'])]
for new_feature in train_test_var.columns.difference(['attack','train_test']):
    train_test_cat[new_feature] = train_test_cat[new_feature].astype('category')
    train_test_cat = handling_dummies(train_test_cat,new_feature)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17312\3336070937.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_cat[new_feature] = train_test_cat[new_feature].astype('category')


In [24]:
train_test_y = train_test_var[['attack','train_test']]

In [25]:
train_test_final = pd.concat([train_test_cat,train_test_num,train_test_y],axis=1)

In [26]:
train_test_final.head()

,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,last_flag,attack,train_test
0,0,0,0,0,0,0,0,0,1,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal,train
1,0,0,0,0,0,0,0,0,1,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal,train
2,0,0,0,0,1,0,0,0,0,0,...,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,neptune,train
3,0,0,0,0,0,0,0,0,1,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal,train
4,0,0,0,0,0,0,0,0,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal,train


In [27]:
train_test_final.columns

Index(['flag_REJ', 'flag_RSTO', 'flag_RSTOS0', 'flag_RSTR', 'flag_S0',
       'flag_S1', 'flag_S2', 'flag_S3', 'flag_SF', 'flag_SH',
       ...
       ' dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       ' dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', ' dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', ' last_flag', 'attack', 'train_test'],
      dtype='object', length=122)

In [28]:
train_test_final['train_test'] = train_test_final['train_test'].apply(lambda x: 1 if x == 'train' else 0)

In [29]:
train_test_final['train_test'].dtype

dtype('int64')

In [30]:
train_test_final = pd.merge(train_test_final,attack_master, left_on = 'attack',right_on = 'attack_type',how = 'left')

In [31]:
train_test_final['attack_binomial'] = train_test_final['attack_class'].apply(lambda x: 0 if x == 'normal' else 1)

In [32]:
train_test_final['attack_binomial']=train_test_final['attack_binomial'].astype('float')

In [33]:
train_test_final['attack_multinomial'] = train_test_final['attack_class'].apply(lambda x: 0 if x == 'normal' else 1 if x == 'DoS' else 2 if x == 'R2L' else 3 if x == 'Probe' else 4)

In [34]:
train_test_final['attack_multinomial']=train_test_final['attack_multinomial'].astype('float')

In [35]:
# Decision trees -  Binomial using Decision Trees

In [36]:
train_bin_new = train_test_final[train_test_final.columns.difference(['attack_type','attack_class','attack_multinomial','attack'])]

In [37]:
train_bin = train_bin_new[(train_bin_new['train_test'] == 1)]

In [38]:
train_bin.shape

(125973, 122)

In [56]:
test_bin = train_bin_new[(train_bin_new['train_test'] == 0)]

In [57]:
test_bin.shape

(22544, 122)

In [45]:
train_bin_x = train_bin[train_bin.columns.difference(['attack_binomial'])]

In [47]:
train_bin_y = train_bin['attack_binomial']

In [48]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV

In [49]:
param_grid = {'max_depth': np.arange(3, 15),
             'max_features': np.arange(3,25)}

In [50]:
tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 10)
tree.fit( train_bin_x, train_bin_y)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'max_features': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24])})

In [51]:
tree.best_params_

{'max_depth': 14, 'max_features': 23}

In [52]:
tree.best_score_

0.9983647265531902

In [53]:
clf_tree = DecisionTreeClassifier( max_depth = 14, max_features=23 )
clf_tree.fit( train_bin_x, train_bin_y )

DecisionTreeClassifier(max_depth=14, max_features=23)

In [58]:
test_bin_x = test_bin[test_bin.columns.difference(['attack_binomial'])]

In [59]:
test_bin_y = test_bin['attack_binomial']

In [60]:
tree_test_pred = pd.DataFrame( { 'actual':  test_bin_y,
                            'predicted': clf_tree.predict( test_bin_x ) } )

In [61]:
metrics.accuracy_score( tree_test_pred.actual, tree_test_pred.predicted )

0.8366305890702626

In [62]:
tree_train_pred = pd.DataFrame( { 'actual':  train_bin_y,
                            'predicted': clf_tree.predict( train_bin_x ) } )

In [63]:
metrics.accuracy_score( tree_train_pred.actual, tree_train_pred.predicted )

0.9996983480587109

In [64]:
import os     
os.environ["PATH"] += os.pathsep + 'C:\Users\Aanchal_gupta\Anaconda\Library\share'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2372498875.py, line 2)

In [65]:
export_graphviz( clf_tree,
              out_file = "hr_tree.odt",
              feature_names = train_bin_x.columns)

In [70]:
import pydotplus as pdot

chd_tree_graph = pdot.graphviz.graph_from_dot_file( 'hr_tree.odt' )

In [71]:
# Decision trees -  Multinomial using Decision Trees

In [72]:
train_multi_new = train_test_final[train_test_final.columns.difference(['attack_type','attack_class','attack_binomial','attack'])]

In [73]:
train_multi = train_multi_new[(train_multi_new['train_test'] == 1)]

In [74]:
train_multi.shape

(125973, 122)

In [75]:
test_multi = train_multi_new[(train_multi_new['train_test'] == 0)]

In [76]:
test_multi.shape

(22544, 122)

In [77]:
train_multi_x = train_multi[train_multi.columns.difference(['attack_multinomial'])]

In [78]:
train_multi_y = train_multi['attack_multinomial']

In [82]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV

In [80]:
param_grid = {'max_depth': np.arange(3, 15),
             'max_features': np.arange(3,25)}

In [81]:
tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 10)
tree.fit( train_multi_x, train_multi_y)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'max_features': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24])})

In [ ]:
tree.best_params_

In [ ]:
tree.best_score_

In [ ]:
clf_tree = DecisionTreeClassifier( max_depth = 14, max_features=24 )
clf_tree.fit( train_multi_x, train_multi_y )

In [ ]:
test_multi_x = test_multi[test_multi.columns.difference(['attack_multinomial'])]

In [ ]:
test_multi_y = test_multi['attack_multinomial']

In [ ]:
tree_test_pred = pd.DataFrame( { 'actual':  test_multi_y,
                            'predicted': clf_tree.predict( test_multi_x ) } )

In [ ]:
metrics.accuracy_score( tree_test_pred.actual, tree_test_pred.predicted )

In [ ]:
tree_train_pred = pd.DataFrame( { 'actual':  train_multi_y,
                            'predicted': clf_tree.predict( train_multi_x ) } )

In [ ]:
metrics.accuracy_score( tree_train_pred.actual, tree_train_pred.predicted )

In [ ]:
export_graphviz( clf_tree,
              out_file = "hr_tree.odt",
              feature_names = train_multi_x.columns)

In [ ]:
import pydotplus as pdot

chd_tree_graph = pdot.graphviz.graph_from_dot_file( 'hr_tree.odt' )